In [1]:
import pandas as pd

loan_data = pd.read_csv('/content/Task 3 and 4_Loan_Data.csv')

##Quantization of FICO Scores into Buckets
###Overview
Quantization is a technique used to reduce the number of continuous values to a smaller set of discrete values or buckets. In the context of FICO scores and default prediction, quantization can help create a more interpretable model by grouping FICO scores into categories (or buckets), each representing a different level of credit risk.

###Objectives
1. **Define Buckets**: Determine the boundaries that summarize the FICO score data, such that the buckets effectively distinguish different levels of credit risk.
2. **Optimize Metrics**: Depending on the objective, choose to minimize the mean squared error or maximize the log-likelihood, which considers both the density of defaults and the roughness of discretization.

###Approaches to Quantization
1. **Equal-Width Binning**: This divides the range of FICO scores into intervals of equal size. It's straightforward but might not capture variations well if the data isn't uniformly distributed.
2. **Equal-Frequency (Quantile) Binning**: This involves dividing the FICO scores so that each bin has approximately the same number of borrowers. This can be more adaptive to the underlying distribution.
3. **Custom Binning Based on Log-Likelihood or MSE**: This approach aims to optimize a specific criterion:
 * Mean Squared Error (MSE) focuses on minimizing the error when approximating all FICO scores in a bucket by the bucket's mean or median.
 * Log-Likelihood maximizes the likelihood of observing the given number of defaults per bucket, which can be particularly useful for ensuring that each bucket is meaningfully differentiated in terms of default probability.


## Implementing Quantization for FICO Scores

###**Step 1: Equal-Frequency Binning**

We'll create a certain number of buckets (let's say 10 for this example) using quantiles of the FICO score distribution. This will help us understand how the FICO scores are distributed across the dataset.

###**Step 2: Custom Binning for Log-Likelihood Optimization**
Based on the distribution observed in Step 1, we'll adjust the bucket boundaries to maximize the log-likelihood, taking into account the number of defaults in each bucket.



###**Step 1: Equal-Frequency Binning**

In [4]:
import numpy as np

#Determine the number of bins (for this task we will use 10)
n_bins = 10

# Use quantile-based binning to create equal-frequency bins for the FICO scores
fico_bins = pd.qcut(loan_data['fico_score'], q=n_bins, duplicates='drop')

# Create a new column in the dataset for the FICO score bins
loan_data['fico_score_bin'] = pd.qcut(loan_data['fico_score'], q=n_bins, duplicates='drop')

# Calculate the number of defaults in each bin
default_summary = loan_data.groupby('fico_score_bin')['default'].agg(['count', 'sum'])

# Calculate the probability of default in each bin
default_summary['default_probability'] = default_summary['sum'] / default_summary['count']

# Display the boundaries of the bins and the default probabilities
default_summary.reset_index(), fico_bins.cat.categories

(     fico_score_bin  count  sum  default_probability
 0  (407.999, 560.0]   1017  499             0.490659
 1    (560.0, 587.0]   1033  318             0.307841
 2    (587.0, 607.0]    987  246             0.249240
 3    (607.0, 623.0]    984  179             0.181911
 4    (623.0, 638.0]    992  173             0.174395
 5    (638.0, 653.0]    997  128             0.128385
 6    (653.0, 670.0]   1027  108             0.105161
 7    (670.0, 688.0]    970   92             0.094845
 8    (688.0, 714.0]   1005   72             0.071642
 9    (714.0, 850.0]    988   36             0.036437,
 IntervalIndex([(407.999, 560.0], (560.0, 587.0], (587.0, 607.0], (607.0, 623.0], (623.0, 638.0], (638.0, 653.0], (653.0, 670.0], (670.0, 688.0], (688.0, 714.0], (714.0, 850.0]], dtype='interval[float64, right]'))

###FICO Score Bins and Default Probabilities:
* **(407.999, 560.0]**: 49.07% default probability
* **(560.0, 587.0]**: 30.78% default probability
* **(587.0, 607.0]**: 24.92% default probability
* **(607.0, 623.0]**: 18.19% default probability
* **(623.0, 638.0]**: 17.44% default probability
* **(638.0, 653.0]**: 12.84% default probability
* **(653.0, 670.0]**: 10.52% default probability
* **(670.0, 688.0]**: 9.48% default probability
* **(688.0, 714.0]**: 7.16% default probability
* **(714.0, 850.0]**: 3.64% default probability

### Observations:
* There is a clear trend where the higher the FICO score, the lower the probability of default. This is expected as higher FICO scores indicate better creditworthiness.
* The bins are well-differentiated in terms of default risk, making them useful for predictive modeling and risk assessment.



### Plan for Implementation:

1. **Preparation**:
   - Sort the data based on FICO scores.
   - Define a function to compute the log-likelihood for a given bucket configuration.

2. **Dynamic Programming Algorithm**:
   - Use a dynamic programming approach to find the optimal partition of the FICO scores into buckets.
   - Define a recursive function \( f(i, b) \) representing the maximum log-likelihood achievable using the first \( i \) scores and \( b \) buckets.
   - The base case and the recursive relation will involve computing the log-likelihood for various possible partitions and choosing the one that maximizes the log-likelihood.

3. **Execution**:
   - Implement the function and run it for the split scenarios:
     - First five buckets from the lowest FICO score up to 600.
     - Next five buckets from 600 to the highest FICO score (850).

4. **Validation and Output**:
   - Validate the output by examining the log-likelihood values and the distribution of defaults within each bucket.
   - Output the optimal bucket boundaries and the corresponding log-likelihood.

Let's start by sorting the data and defining the function to compute the log-likelihood for any given set of bucket boundaries. We'll also look at separating the FICO scores into two main ranges

In [7]:
# Sort the data based on FICO scores
sorted_loan_data = loan_data.sort_values(by='fico_score')

# Function to compute log-likelihood for a given set of buckets
def compute_log_likelihood(data):
    if len(data) == 0 or data['default'].sum() == 0 or data['default'].sum() == len(data):
        return 0  # Avoid log(0) issues by returning 0 likelihood in degenerate cases.
    p = data['default'].mean()
    k = data['default'].sum()
    n = len(data)
    return k * np.log(p) + (n - k) * np.log(1 - p)

# Initial split as suggested: two ranges, 0-600 and 600-850
# We need to adapt the range since the actual minimum FICO score in the data is 408
range1 = sorted_loan_data[sorted_loan_data['fico_score'] <= 600]
range2 = sorted_loan_data[sorted_loan_data['fico_score'] > 600]

# Log-likelihood of initial simple splits
ll_range1 = compute_log_likelihood(range1)
ll_range2 = compute_log_likelihood(range2)

ll_range1, ll_range2, ll_range1 + ll_range2

(-1756.162014648322, -2686.830271064601, -4442.992285712923)

In [10]:
# Filter data to focus on the range of interest and reset index for easier handling
dp_data = sorted_loan_data[(sorted_loan_data['fico_score'] >= 400) & (sorted_loan_data['fico_score'] <= 600)].copy()
dp_data.reset_index(drop=True, inplace=True)

# Calculate cumulative defaults and totals for each FICO score
max_fico = 600
min_fico = 400
fico_range = max_fico - min_fico + 1

# Initialize lists for cumulative sums
defaults = [0] * fico_range
totals = [0] * fico_range

# Fill in the defaults and totals from the data
for index, row in dp_data.iterrows():
    fico_index = row['fico_score'] - min_fico  # Adjust index to start from 0
    defaults[fico_index] += row['default']
    totals[fico_index] += 1

# Convert to cumulative sums
for i in range(1, fico_range):
    defaults[i] += defaults[i - 1]
    totals[i] += totals[i - 1]

# Log-likelihood calculation function
def log_likelihood(n, k):
    if n == 0 or k == 0 or k == n:
        return 0
    p = k / n
    return k * np.log(p) + (n - k) * np.log(1 - p)

# Setting up the dynamic programming array
r = 10  # Number of buckets
dp = [[[-float('inf'), 0] for _ in range(fico_range)] for _ in range(r + 1)]

# Initialize DP for 0 buckets, 0 log-likelihood
for j in range(fico_range):
    dp[0][j][0] = 0

# Fill the DP table
for i in range(1, r + 1):
    for j in range(fico_range):
        for k in range(j + 1):  # Allow split at each possible score
            total_k_to_j = totals[j] - (totals[k - 1] if k > 0 else 0)
            default_k_to_j = defaults[j] - (defaults[k - 1] if k > 0 else 0)
            ll = log_likelihood(total_k_to_j, default_k_to_j)
            if i == 1:
                potential_ll = ll
            else:
                potential_ll = dp[i - 1][k][0] + ll

            if potential_ll > dp[i][j][0]:
                dp[i][j][0] = potential_ll
                dp[i][j][1] = k

# Extract the boundaries from the DP table
boundaries = []
last_index = fico_range - 1
for i in range(r, 0, -1):
    boundaries.append(last_index + min_fico)
    last_index = dp[i][last_index][1] - 1

# Log-likelihood of the optimal configuration
optimal_ll = dp[r][fico_range - 1][0]
optimal_ll, boundaries[::-1]  # Return boundaries in ascending order of scores


(-297.9889211260869, [589, 590, 591, 592, 593, 594, 595, 597, 599, 600])

In [11]:
# Filter data to focus on the range of interest and reset index for easier handling
dp_data = sorted_loan_data[(sorted_loan_data['fico_score'] >= 601) & (sorted_loan_data['fico_score'] <= 850)].copy()
dp_data.reset_index(drop=True, inplace=True)

# Calculate cumulative defaults and totals for each FICO score
max_fico = 850
min_fico = 601
fico_range = max_fico - min_fico + 1  # 250 possible FICO scores within the range

# Initialize lists for cumulative sums
defaults = [0] * fico_range
totals = [0] * fico_range

# Fill in the defaults and totals from the data
for index, row in dp_data.iterrows():
    fico_index = row['fico_score'] - min_fico  # Adjust index to start from 0
    defaults[fico_index] += row['default']
    totals[fico_index] += 1

# Convert to cumulative sums
for i in range(1, fico_range):
    defaults[i] += defaults[i - 1]
    totals[i] += totals[i - 1]

# Log-likelihood calculation function
def log_likelihood(n, k):
    if n == 0 or k == 0 or k == n:
        return 0
    p = k / n
    return k * np.log(p) + (n - k) * np.log(1 - p)

# Setting up the dynamic programming array
r = 10  # Number of buckets
dp = [[[-float('inf'), 0] for _ in range(fico_range)] for _ in range(r + 1)]

# Initialize DP for 0 buckets, 0 log-likelihood
for j in range(fico_range):
    dp[0][j][0] = 0

# Fill the DP table
for i in range(1, r + 1):
    for j in range(fico_range):
        for k in range(j + 1):  # Allow split at each possible score
            total_k_to_j = totals[j] - (totals[k - 1] if k > 0 else 0)
            default_k_to_j = defaults[j] - (defaults[k - 1] if k > 0 else 0)
            ll = log_likelihood(total_k_to_j, default_k_to_j)
            if i == 1:
                potential_ll = ll
            else:
                potential_ll = dp[i - 1][k][0] + ll

            if potential_ll > dp[i][j][0]:
                dp[i][j][0] = potential_ll
                dp[i][j][1] = k

# Extract the boundaries from the DP table
boundaries = []
last_index = fico_range - 1
for i in range(r, 0, -1):
    boundaries.append(last_index + min_fico)
    last_index = dp[i][last_index][1] - 1

# Log-likelihood of the optimal configuration
optimal_ll = dp[r][fico_range - 1][0]
optimal_ll, boundaries[::-1]  # Return boundaries in ascending order of scores


(0, [755, 763, 765, 772, 774, 787, 788, 808, 810, 850])

In [12]:
# Manual checks for specific segments between the derived bucket boundaries
# Extracted boundaries: [755, 763, 765, 772, 774, 787, 788, 808, 810, 850]

# Define boundaries in a list for ease of iteration
boundaries = [589, 590, 591, 592, 593, 594, 595, 597, 599, 600]

# Prepare to display data from specific segments
validation_results = []
for i in range(len(boundaries) - 1):
    start = boundaries[i] - min_fico
    end = boundaries[i + 1] - min_fico
    total_k_to_j = totals[end] - (totals[start - 1] if start > 0 else 0)
    default_k_to_j = defaults[end] - (defaults[start - 1] if start > 0 else 0)
    ll = log_likelihood(total_k_to_j, default_k_to_j)
    validation_results.append({
        'start_fico': boundaries[i],
        'end_fico': boundaries[i + 1],
        'total_loans': total_k_to_j,
        'defaults': default_k_to_j,
        'log_likelihood': ll
    })

# Display the results for validation
pd.DataFrame(validation_results)


,start_fico,end_fico,total_loans,defaults,log_likelihood
0,589,590,7317,879,-2686.702298
1,590,591,7317,879,-2686.702298
2,591,592,7317,879,-2686.702298
3,592,593,7317,879,-2686.702298
4,593,594,7317,879,-2686.702298
5,594,595,7317,879,-2686.702298
6,595,597,7317,879,-2686.702298
7,597,599,7317,879,-2686.702298
8,599,600,7318,879,-2686.830271
